In [ ]:
# imports
import numpy as np
import pandas as pd
import json

In [ ]:
country_map = {
    "nl": "Netherlands",
    "fr": "France",
    "dk": "Denmark",
    "lu": "Luxembourg",
    "de": "Germany",
    "it": "Italy",
    "gb": "United Kingdom",
    "be": "Belgium",
    "ch": "Switzerland",
    "at": "Austria",
    "se": "Sweden",
    "mc": "Monaco",
    "no": "Norway",
    "yu": "Yugoslavia",
    "es": "Spain",
    "fi": "Finland",
    "pt": "Portugal",
    "ie": "Ireland",
    "mt": "Malta",
    "il": "Israel",
    "gr": "Greece",
    "tr": "Turkey",
    "ma": "Morocco",
    "cy": "Cyprus",
    "is": "Iceland",
    "hr": "Croatia",
    "ba": "Bosnia and Herzegovina",
    "si": "Slovenia",
    "pl": "Poland",
    "hu": "Hungary",
    "ru": "Russia",
    "sk": "Slovakia",
    "ro": "Romania",
    "ee": "Estonia",
    "lt": "Lithuania",
    "mk": "North Macedonia",
    "lv": "Latvia",
    "ua": "Ukraine",
    "cs": "Serbia and Montenegro",
    "al": "Albania",
    "ad": "Andorra",
    "by": "Belarus",
    "md": "Moldova",
    "bg": "Bulgaria",
    "am": "Armenia",
    "rs": "Serbia",
    "ge": "Georgia",
    "me": "Montenegro",
    "cz": "Czech Republic",
    "az": "Azerbaijan",
    "sm": "San Marino",
    "au": "Australia"
}

votes_df = pd.read_csv('votes.csv')
votes_df['to_country'] = votes_df['to_country'].map(country_map)
votes_df['from_country'] = votes_df['from_country'].map(country_map)

In [ ]:
incoming_votes_df = votes_df[votes_df['round'] == 'final'].drop(columns=['round', 'from_country_id', 'from_country', 'to_country_id'])
incoming_votes_df = incoming_votes_df.groupby(['year', 'to_country']).sum().fillna(0)
incoming_votes_df

In [ ]:
max_votes_df = incoming_votes_df.groupby('year').max()
max_votes_df

In [ ]:
nested_json = {}

for outer_key, group_df in incoming_votes_df.groupby(level=0):
    inner = group_df.droplevel(0)
    nested_json[outer_key] = inner.to_dict(orient="index")

In [ ]:
result_json = {}

for year, countries in nested_json.items():
    countries_json = {}
    for country, points in countries.items():
        if max_votes_df.loc[year]['tele_points'] != 0 and max_votes_df.loc[year]['jury_points'] != 0 :
            votes = [int(points['total_points']), int(points['tele_points']), int(points['jury_points'])]
            normalized = [points['total_points'] / max_votes_df.loc[year]['total_points'],
                          points['tele_points'] / max_votes_df.loc[year]['tele_points'], 
                          points['jury_points'] / max_votes_df.loc[year]['jury_points']]
        else:
            votes = [int(points['total_points'])]
            normalized = [points['total_points'] / max_votes_df.loc[year]['total_points']]

        
        countries_json[country] = {'normalized': normalized, 'votes': votes}

    result_json[year] = countries_json

In [ ]:
with open("color_map_data.json", "w") as f:
    json.dump(result_json, f, indent=4)